In [60]:
import numpy as np
import pandas as pd

In [61]:
# load labels
labels = pd.read_csv('../data/Lauren/Labels.csv')
labels.head() # to display the first 5 lines of loaded data

,Class,Subclass,cluster
0,GABAergic,Vip,Vip Arhgap36 Hmcn1
1,GABAergic,Lamp5,Lamp5 Lsp1
2,GABAergic,Lamp5,Lamp5 Lsp1
3,GABAergic,Vip,Vip Crispld2 Htr2c
4,GABAergic,Lamp5,Lamp5 Plch2 Dock5


In [62]:
# load data
import time
start = time.time()
data = pd.read_csv('../data/Lauren/Filtered_mouse_allen_brain_data-100.csv')
print(time.time() - start)
data.head() # to display the first 5 lines of loaded data

14.858144044876099


,Unnamed: 0,0610005C13Rik,0610006L08Rik,0610007P14Rik,0610009B22Rik,0610009E02Rik,0610009L18Rik,0610009O20Rik,0610010B08Rik,0610010F05Rik,...,Zxdb,Zxdc,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,a,l7Rn6,n-R5s136
0,F1S4_160108_001_A01,0,0,79,145,1,46,123,0,0,...,0,0,0,78,0,36,1,0,190,0
1,F1S4_160108_001_B01,0,0,123,178,26,23,2,0,337,...,0,0,0,69,0,78,29,0,179,0
2,F1S4_160108_001_C01,0,0,89,69,0,13,29,0,112,...,0,0,2,273,38,110,81,0,178,0
3,F1S4_160108_001_D01,0,0,115,0,0,0,50,0,35,...,0,0,0,151,165,67,30,0,110,0
4,F1S4_160108_001_E01,0,0,402,208,0,42,0,0,0,...,20,0,1,99,50,83,69,0,335,0


In [63]:
## Filter labels
filter_on = "Class"
amount_higher_than = 10


classes_count = labels.groupby(filter_on).count()
print(classes_count)
print()
# the count is in last column: classes_count[classes_count.columns[-1]
classes_to_keep = list(classes_count[classes_count[classes_count.columns[-1]] >= amount_higher_than].index)
print(classes_to_keep)
print()
indices = labels[filter_on].isin(classes_to_keep)

labels_filtered = labels[indices]
print(labels.shape)
print(labels_filtered.shape)



               Subclass  cluster
Class                           
Endothelial           6        6
GABAergic          5643     5643
Glutamatergic      7155     7155
Non-Neuronal         28       28

['GABAergic', 'Glutamatergic', 'Non-Neuronal']

(12832, 3)
(12826, 3)


In [58]:
## Filter data
